In [1]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np
from math import ceil

from simulation import run_simulation
from graph_create import const_kregular, reduce_providers_simplest

init_notebook_mode(connected=True)

In [2]:
def plot_utils(p_utils, r):
    fig = go.Figure()

    red_vals = np.linspace(200, 0, 10, endpoint=False)
    for p_idx, (p, utils) in enumerate(p_utils):
        x_vals = list(range(len(utils)))
        rgb = 'rgb({0}, 70, 200)'
        rgb_col = rgb.format(red_vals[p_idx])
        p_hover = 'p={0}'.format(p)
        fig.add_trace(go.Scatter(x=x_vals,
            y=utils, mode='lines+markers', hoverinfo='text',
            hovertext=p_hover,
            marker=dict(color=rgb_col)))
    fig.layout.update(showlegend=False)
    title_fmt = "Social Welfare vs Iteration (r={0})"
    plot_title = title_fmt.format(r)
    fig.layout.update(title=plot_title)

    iplot(fig)

In [3]:
# Constants
num_vertices = 100
regularity = 4

In [4]:
for r in np.linspace(100, 0, 10, endpoint=False):
    util_set = []
    for p in np.linspace(1, 0, 10, endpoint=False):
        ring_lat = const_kregular(regularity, num_vertices, p, r)
        simp_ring_lat = reduce_providers_simplest(ring_lat)
        
        sim_g, sim_utils = run_simulation(ring_lat)
        
        global_util = [ sum(it_utils) for it_utils in sim_utils ]
        
        util_set.append((p, global_util))
        
    plot_utils(util_set, r)


In [5]:
def plot_conv(conv_rates, r):
    fig = go.Figure()

    prk_vals = [ p * r / regularity for (p, cr) in conv_rates ]
    cr_vals = [ cr for (p, cr) in conv_rates ]
    fig.add_trace(go.Scatter(x=prk_vals,
        y=cr_vals, mode='lines+markers', hoverinfo='skip'))
        
    fig.layout.update(showlegend=False)
    title_fmt = "Average Convergance Rate vs p*r/k (r={0})"
    plot_title = title_fmt.format(r)
    fig.layout.update(title=plot_title)

    iplot(fig)

In [6]:
for r in np.linspace(100, 0, 10, endpoint=False):
    conv_rates = []
    for p in np.linspace(1, 0, 10, endpoint=False):
        tot_conv_rates = 0
        num_iter = 10
        for i in range(num_iter):
            ring_lat = const_kregular(regularity, num_vertices, p, r)
            simp_ring_lat = reduce_providers_simplest(ring_lat)

            sim_g, sim_utils = run_simulation(ring_lat)
            
            if sum(sim_utils[-1]) != len(sim_utils[-1]):
                tot_conv_rates += -1 * len(sim_utils)
            else:
                tot_conv_rates += len(sim_utils)
        avg_conv_rate = tot_conv_rates / num_iter
        conv_rates.append((p, avg_conv_rate))
        
    plot_conv(conv_rates, r)

In [7]:
fixed_prk_rates = []
for r in np.linspace(100, 0, 100, endpoint=False):
    p = ceil(regularity / r)
    
    tot_conv_rates = 0
    num_iter = 20
    
    for i in range(num_iter):
        ring_lat = const_kregular(regularity, num_vertices, p, r)
        simp_ring_lat = reduce_providers_simplest(ring_lat)

        sim_g, sim_utils = run_simulation(ring_lat)

        if sum(sim_utils[-1]) != len(sim_utils[-1]):
            tot_conv_rates += -1 * len(sim_utils)
        else:
            tot_conv_rates += len(sim_utils)
            
    avg_conv_rate = tot_conv_rates / num_iter
    fixed_prk_rates.append((r, avg_conv_rate))

In [8]:
def plot_fixed_one(conv_rates):
    fig = go.Figure()

    r_vals = [ r for (r, cr) in conv_rates ]
    cr_vals = [ cr for (r, cr) in conv_rates ]
    fig.add_trace(go.Scatter(x=r_vals,
        y=cr_vals, mode='lines+markers', hoverinfo='skip'))
        
    fig.layout.update(showlegend=False)
    plot_title = "Average Convergance Rate vs r (p*r/k=1)"
    fig.layout.update(title=plot_title)

    iplot(fig)

plot_fixed_one(fixed_prk_rates)